In [1]:
import pandas as pd
import string
!pip install nltk

In [2]:
from nltk.corpus import stopwords

In [3]:
df_spam=pd.read_csv('SpamCollection',sep='\t',names=['response','message'])

In [4]:
df_spam.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df_spam.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
df_spam.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [7]:
df_spam['length']=df_spam['message'].apply(len)

In [8]:
df_spam.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [9]:
def message_text_process(mess):
    no_punctuation=[char for char in mess if char not in string.punctuation]
    no_punctuation=''.join(no_punctuation)
    return[word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [10]:
df_spam['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
bag_of_words=CountVectorizer(analyzer=message_text_process).fit(df_spam['message'])

In [13]:
print(len(bag_of_words.vocabulary_))

11425


In [14]:
message_bagofwords=bag_of_words.transform(df_spam['message'])

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(message_bagofwords)

In [17]:
message_tfidf=tfidf_transformer.transform(message_bagofwords)
print(message_tfidf.shape)

(5572, 11425)


In [18]:
from sklearn.naive_bayes import MultinomialNB
spam_det_model=MultinomialNB().fit(message_tfidf,df_spam['response'])

In [22]:
message=df_spam['message'][5]
bag_of_words_message=bag_of_words.transform([message])
tfidf=tfidf_transformer.transform(bag_of_words_message)

print('predicted ',spam_det_model.predict(tfidf)[0])
print('expected ',df_spam.response[0])

predicted  ham
expected  ham
